In [1]:
def create_gold_combinations(classes, sentiments):
    r = []
    for c in classes:
        for s in sentiments:
            r.append((c, s))
            
    return r
create_gold_combinations(['a', 'b'], ['n/a', 'neg', 'pos', 'neu'])

[('a', 'n/a'),
 ('a', 'neg'),
 ('a', 'pos'),
 ('a', 'neu'),
 ('b', 'n/a'),
 ('b', 'neg'),
 ('b', 'pos'),
 ('b', 'neu')]

In [14]:
# 'def create_eval_entries_from_gold_and_predictions(golds, predictions):
#     eval_entries = []
#     for gold_doc, pred_doc in zip(golds, predictions):
#         # only works for the small example where only sentiment is misclassified
#         for (g_cls, g_sent), (p_cls, p_sent) in zip(gold_doc, pred_doc):
#             if g_cls
#         eval_entries.extend(zip(gold_doc, pred_doc))
#     return eval_entries

# golds = [
#     [ # Doc 1
#         ('a', 'neg'),
#     ], 
#     [ # Doc 2
#         ('a', 'pos'),
#         ('b', 'pos')
#     ]
# ]

# predictions = [
#     [ # Doc 1
#         ('a', 'neg'),
#     ], 
#     [ # Doc 2
#         ('a', 'pos'),
#         ('b', 'negative')
#     ]
# ]

# eval_entries = create_eval_entries_from_gold_and_predictions(golds, predictions)

# eval_entries'
eval_entries = [
    (('a', 'neu'),('a', 'neu')),
    (('a', 'neu'), ('a', 'neu')),
    (('b', 'neu'), ('b', 'n/a'))
    #(('b', 'n/a'), ('b', 'pos'))
]

In [15]:
def get_metrics(classes, sentiments, eval_entries):
    combinations = create_gold_combinations(classes, sentiments)
    
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    
    for cls, sent in combinations:
        for (g_cls, g_sent), (p_cls, p_sent) in eval_entries:
            if g_cls == cls and g_sent == sent:
                if g_cls == p_cls and g_sent == p_sent:
                    tp += 1
                else:
                    fn += 1
            else:
                if p_cls == cls and p_sent == sent:
                    fp += 1
                else:
                    tn += 1
                    
    return (tp, fp, fn, tn)

get_metrics(['a', 'b'], ['n/a', 'neg', 'pos', 'neu'], eval_entries)

(2, 1, 1, 20)

In [16]:
def calculate_micro_f1(classes, sentiments, eval_entries):
    tp, fp, fn, _ = get_metrics(classes, sentiments, eval_entries)
    micro_precission = tp / (tp + fp)
    micro_recall = tp / (tp + fn)
    micro_f1 = 2.0 * micro_precission * micro_recall / (micro_precission + micro_recall)
    return micro_f1

calculate_micro_f1(['a', 'b'], ['n/a', 'neg', 'pos', 'neu'], eval_entries)

0.6666666666666666